# Prevendo roubo de celulares em SP

## Baixando biblioteca necessária
`pip install xlwt`

`conda install xlwt`

### Importando bibliotecas Pandas e Numpy

In [25]:
import pandas as pd
import numpy as np

### Abrindo arquivo XLS disponível no SSP-SP

#### Os arquivos possuem um erro, sendo impossível abrir o arquivo
*XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'A\x00N\x00O\x00_\x00'*

A solução foi encontrada nesse artigo do [Medium](https://medium.com/@jerilkuriakose/recover-corrupt-excel-file-xls-or-xlsx-using-python-2eea6bb07aae)

#### Apenas para resolver o problema de formato
Você pode até salvar o arquivo como .csv, mas a leitura pode informar algum erro

In [22]:
from __future__ import unicode_literals
# Used to save the file as excel workbook
# Need to install this library
from xlwt import Workbook
# Used to open to corrupt excel file
import io

filename = 'DadosBO_2020_8.xls'
# Opening the file using 'utf-16' encoding
file = io.open(filename, "r", encoding="utf-16-le")
data = file.readlines()

# Creating a workbook object
xldoc = Workbook()
# Adding a sheet to the workbook object
sheet = xldoc.add_sheet("Sheet1", cell_overwrite_ok=True)
# Iterating and saving the data to sheet
for i, row in enumerate(data):
    # Two things are done here
    # Removeing the '\n' which comes while reading the file using io.open
    # Getting the values after splitting using '\t'
    for j, val in enumerate(row.replace('\n', '').split('\t')):
        sheet.write(i, j, val)
    
# Saving the file as an excel file
xldoc.save('file.xls')

In [23]:
#dataset = pd.read_excel("DadosBO_2020_7.xls", engine='xlrd')
columns = ['ANO_BO', 'NUM_BO', 'BO_INICIADO', 'BO_EMITIDO', 'DATAOCORRENCIA', 'HORAOCORRENCIA', 'PERIDOOCORRENCIA', 'DATACOMUNICACAO', 'DATAELABORACAO', 'BO_AUTORIA', 'FLAGRANTE', 'LOGRADOURO', 'NUMERO', 'BAIRRO', 'CIDADE', 'UF', 'LATITUDE', 'LONGITUDE', 'DESCRICAOLOCAL', 'SOLUCAO', 'DELEGACIA_NOME', 'DELEGACIA_CIRCUNSCRICAO', 'ESPECIE', 'RUBRICA', 'DESDOBRAMENTO', 'STATUS', 'QUANT_CELULAR', 'MARCA_CELULAR']

column_dates = ['BO_INICIADO', 'BO_EMITIDO', 'DATAOCORRENCIA', 'HORAOCORRENCIA','DATACOMUNICACAO', 'DATAELABORACAO']

dtypes = {
    'ANO_BO': np.int32,
    'NUM_BO': np.int64,
    'QUANT_CELULAR': pd.Int16Dtype(),
    'LATITUDE': np.float32,
    'LONGITUDE': np.float32 
}

dataset = pd.read_excel("file.xls", parse_dates=column_dates, usecols=columns, dtype=dtypes, thousands=',')

In [26]:
dataset

,ANO_BO,NUM_BO,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,...,DESCRICAOLOCAL,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,QUANT_CELULAR,MARCA_CELULAR
0,2020,1069122,2020-01-08 00:09:32,2020-01-08 00:07:52,2020-07-28,2020-09-30 11:45:00,PELA MANHÃ,2020-07-31,2020-01-08 00:09:32,Desconhecida,...,Via Pública,BO PARA REGISTRO,DELEGACIA ELETRONICA,62º D.P. ERMELINO MATARAZZO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,<NA>,LG
1,2020,1069823,2020-07-31 23:25:40,2020-01-08 00:10:55,2020-07-31,2020-09-30 12:53:00,A TARDE,2020-07-31,2020-07-31 23:25:40,Desconhecida,...,Via pública,ENCAMINHAMENTO DP ÁREA DO FATO,DELEGACIA ELETRONICA,05º D.P. ACLIMACAO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - TRANSEUNTE,NaN,Consumado,<NA>,APPLE
2,2020,1069786,2020-07-31 23:12:57,2020-01-08 00:16:51,2020-07-31,2020-09-30 16:55:00,A TARDE,2020-07-31,2020-07-31 23:12:57,Desconhecida,...,Via pública,ENCAMINHAMENTO DP ÁREA DO FATO,DELEGACIA ELETRONICA,54º D.P. CID. TIRADENTES,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - TRANSEUNTE,NaN,Consumado,<NA>,SAMSUNG
3,2020,1495,2020-07-31 21:34:49,2020-01-08 00:17:43,2020-07-31,2020-09-30 21:00:00,A NOITE,2020-07-31,2020-07-31 21:34:49,Conhecida,...,Via pública,BO PARA FLAGRANTE,DEL.SEC.JACAREI PLANTÃO,02º D.P. JACAREI,Ato infracional,A.I.-Roubo (art. 157) - VEICULO,NaN,Consumado,1,Motorola
4,2020,1495,2020-07-31 21:34:49,2020-01-08 00:17:43,2020-07-31,2020-09-30 21:00:00,A NOITE,2020-07-31,2020-07-31 21:34:49,Conhecida,...,Via pública,BO PARA FLAGRANTE,DEL.SEC.JACAREI PLANTÃO,02º D.P. JACAREI,Ato infracional,A.I.-Roubo (art. 157) - VEICULO,NaN,Consumado,1,SAMSUNG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13527,2020,5342,2020-08-31 23:36:15,2020-08-31 23:48:52,2020-08-31,2020-09-30 22:00:00,A NOITE,2020-08-31,2020-08-31 23:36:15,Desconhecida,...,Via pública,APRECIAÇÃO DO DELEGADO TITULAR,DEL.SEC.RIO CLARO PLANTÃO,03º D.P. RIO CLARO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,1,MOTOROLA
13528,2020,4656,2020-08-31 23:31:17,2020-08-31 23:52:29,2020-08-31,2020-09-30 19:10:00,A NOITE,2020-08-31,2020-08-31 23:31:17,Desconhecida,...,Via pública,BO PARA INVESTIGAÇÃO,101º D.P. JDIM IMBUIAS,101º D.P. JDIM IMBUIAS,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - TRANSEUNTE,NaN,Consumado,1,Samsung
13529,2020,1255538,2020-08-31 21:10:54,2020-08-31 23:55:22,2020-08-31,2020-09-30 15:00:00,A TARDE,2020-08-31,2020-08-31 21:10:54,Desconhecida,...,Via Pública,BO PARA REGISTRO,DELEGACIA ELETRONICA,03º D.P. CUBATÃO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - ESTABELECIMENTO-OUTROS,NaN,Consumado,1,MOTOROLA
13530,2020,1255308,2020-08-31 23:57:57,2020-08-31 23:57:52,2020-08-31,2020-09-30 19:30:00,A NOITE,2020-08-31,2020-08-31 23:57:57,Desconhecida,...,Garagem ou abrigo de residência,BO PARA REGISTRO,DELEGACIA ELETRONICA,77º D.P. SANTA CECILIA,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,<NA>,XIAOMI


In [85]:
dataset.dtypes

ANO_BO                              int32
NUM_BO                              int64
BO_INICIADO                datetime64[ns]
BO_EMITIDO                 datetime64[ns]
DATAOCORRENCIA             datetime64[ns]
HORAOCORRENCIA             datetime64[ns]
PERIDOOCORRENCIA                   object
DATACOMUNICACAO            datetime64[ns]
DATAELABORACAO             datetime64[ns]
BO_AUTORIA                         object
FLAGRANTE                          object
LOGRADOURO                         object
NUMERO                            float64
BAIRRO                             object
CIDADE                             object
UF                                 object
LATITUDE                          float32
LONGITUDE                         float32
DESCRICAOLOCAL                     object
SOLUCAO                            object
DELEGACIA_NOME                     object
DELEGACIA_CIRCUNSCRICAO            object
ESPECIE                            object
RUBRICA                           

In [65]:
max(dataset.NUMERO)

1399738354.0

#### Capturando nome das colunas importantes no dataset

In [39]:
string_gerada=""
for item in dataset.columns:
    string_gerada += "'"+item+"'"+", "
string_gerada

"'ANO_BO', 'NUM_BO', 'NUMERO_BOLETIM', 'BO_INICIADO', 'BO_EMITIDO', 'DATAOCORRENCIA', 'HORAOCORRENCIA', 'PERIDOOCORRENCIA', 'DATACOMUNICACAO', 'DATAELABORACAO', 'BO_AUTORIA', 'FLAGRANTE', 'NUMERO_BOLETIM_PRINCIPAL', 'LOGRADOURO', 'NUMERO', 'BAIRRO', 'CIDADE', 'UF', 'LATITUDE', 'LONGITUDE', 'DESCRICAOLOCAL', 'EXAME', 'SOLUCAO', 'DELEGACIA_NOME', 'DELEGACIA_CIRCUNSCRICAO', 'ESPECIE', 'RUBRICA', 'DESDOBRAMENTO', 'STATUS', 'NOMEPESSOA', 'TIPOPESSOA', 'VITIMAFATAL', 'RG', 'RG_UF', 'NATURALIDADE', 'NACIONALIDADE', 'SEXO', 'DATANASCIMENTO', 'IDADE', 'ESTADOCIVIL', 'PROFISSAO', 'GRAUINSTRUCAO', 'CORCUTIS', 'NATUREZAVINCULADA', 'TIPOVINCULO', 'RELACIONAMENTO', 'PARENTESCO', 'PLACA_VEICULO', 'UF_VEICULO', 'CIDADE_VEICULO', 'DESCR_COR_VEICULO', 'DESCR_MARCA_VEICULO', 'ANO_FABRICACAO', 'ANO_MODELO', 'DESCR_TIPO_VEICULO', 'QUANT_CELULAR', 'MARCA_CELULAR', "

In [42]:
drop_c = ['ANO_BO', 'NUM_BO', 'BO_INICIADO', 'BO_EMITIDO', 'DATAOCORRENCIA', 'HORAOCORRENCIA', 'PERIDOOCORRENCIA', 'DATACOMUNICACAO', 'DATAELABORACAO', 'BO_AUTORIA', 'FLAGRANTE', 'LOGRADOURO', 'NUMERO', 'BAIRRO', 'CIDADE', 'UF', 'LATITUDE', 'LONGITUDE', 'DESCRICAOLOCAL', 'SOLUCAO', 'DELEGACIA_NOME', 'DELEGACIA_CIRCUNSCRICAO', 'ESPECIE', 'RUBRICA', 'DESDOBRAMENTO', 'STATUS', 'QUANT_CELULAR', 'MARCA_CELULAR']

In [ ]:
string_gerada=[]
for item in dataset.columns:
    string_gerada.append(item)
string_gerada

In [47]:
columns = ['NUMERO_BOLETIM', 'NUMERO_BOLETIM_PRINCIPAL', 'EXAME', 'NOMEPESSOA', 'TIPOPESSOA', 'VITIMAFATAL', 'RG', 'RG_UF', 'NATURALIDADE', 'NACIONALIDADE', 'SEXO', 'DATANASCIMENTO', 'IDADE', 'ESTADOCIVIL', 'PROFISSAO', 'GRAUINSTRUCAO', 'CORCUTIS', 'NATUREZAVINCULADA', 'TIPOVINCULO', 'RELACIONAMENTO', 'PARENTESCO', 'PLACA_VEICULO', 'UF_VEICULO', 'CIDADE_VEICULO', 'DESCR_COR_VEICULO', 'DESCR_MARCA_VEICULO', 'ANO_FABRICACAO', 'ANO_MODELO', 'DESCR_TIPO_VEICULO']

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
